### Libraries

In [ ]:
import cartopy.crs as ccrs
import fsspec
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import planetary_computer
import pystac_client
import xarray as xr
import geopandas as gpd

In [ ]:
planetary_computer.set_subscription_key("c27669c4bdec434d804e2bd738cb16fc")
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)
print(catalog)

### Gather User Input

Assumptions:
1. User can input coordinates that respresent a region of interest ROI
2. User can input names for common locations (countries, cities, states etc)
3. The shape of the user input will be a four sided polygon (for now do not worry about multisided polygons)

In [ ]:
#longitude = 79.109
#latitude = 22.746

#GeoJSON - East Melbourne (square)
geometry = {
    "type": "Polygon",
    "coordinates": [    
        [
            [-37.7528,145.096],
            [-37.75415,144.85276],
            [-37.92253,144.84092],
            [-37.92551,145.09532],
            [-37.7528,145.096],
        ]
    ],
}

#name
user_countries = "Australia" 

#user_cities = "Melbourne"

#user_states = "Victoria"

#continent
user_continent = "Oceania"


In [ ]:
from datetime import datetime, time

# Function to convert date format 
def convert_format_date(input_date, input_format, output_format):
    date_obj = datetime.strptime(input_date, input_format)
    formatted_date = date_obj.strftime(output_format)
    return formatted_date

# User Input Date and time
user_start_date = "1 09 2023"
user_end_date = "01-September-2023"

user_start_time = time(14,30,0)
user_end_time = time(15,00,0)

# Define input and output date formats
input_format = "%d %m %Y"
output_format = "%d/%m/%Y"

# Convert user start date format
user_start_date1 = convert_format_date(user_start_date, input_format, output_format)
print(user_start_date1)

# Convert user end date format
input_format = "%d-%B-%Y" 
user_end_date1 = convert_format_date(user_end_date, input_format, output_format)
print(user_end_date1)

print(user_start_time)
print(user_end_time)

In [ ]:

import json
from turfpy.measurement import bbox_polygon, bbox
from geojson import Polygon

#Bounding box - Swinburne University Hawthorn campus (square)
#minLat: -37.822978
#minLon: 145.037186
#maxLat: -37.821866
#maxLon: 145.038841
bbox = [-37.822978, 145.037186, -37.821866, 145.038841]
print(json.dumps(bbox_polygon(bbox), indent=2, sort_keys=True))#json.dumps() function will convert a subset of Python objects into a json string.


In [ ]:
world_data = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
capitals = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))
specific_city = capitals[capitals['name'] == user_cities]

capitals

In [ ]:
world_data.plot()
specific_city.plot()

### Data query

In [ ]:
search = catalog.search(
    collections="sentinel-5p-l2-netcdf",
    intersects=geometry,
    datetime="2023-04-03/2023-04-04",
    query={"s5p:processing_mode": {"eq": "OFFL"}, "s5p:product_name": {"eq": "ch4"}},
)

items = list(search.get_items())

print(f"Found {len(items)} items:")

for item in search.items():
    print(item.id)

### Reading and Processing Data

In [ ]:
# Read data
f = fsspec.open(items[0].assets["ch4"].href).open()
ds = xr.open_dataset(f, group="PRODUCT", engine="h5netcdf") # Putting the data into a xarray dataset
ds

### Plot Base Map

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
australia = world[world['continent'] == user_continent]
australia.plot()

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
australia = world[world['name'] == user_countries]
australia.plot()

### Plot Concentration Values

### Time Series

In [ ]:
varname = "methane_mixing_ratio_bias_corrected"
data = ds[varname][0, :, :]
vmin, vmax = np.nanpercentile(data, [1, 99])
data.plot(vmin=vmin, vmax=vmax, cmap="viridis")

In [ ]:
lon = ds["longitude"].values.squeeze()
lat = ds["latitude"].values.squeeze()
methane = data.values

fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
ax.coastlines()
ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, alpha=0.5, linestyle="--")
ax.set_extent([-180, 180, -90, 90], crs=ccrs.PlateCarree())
norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
scatter = plt.scatter(
    lon,
    lat,
    c=methane,
    transform=ccrs.PlateCarree(),
    cmap="viridis",
    norm=norm,
    marker=".",
    s=1,
)
fig.colorbar(scatter, pad=0.05, shrink=0.35, label="Methane parts/billion)")
plt.show()